In [11]:
import numpy as np
import pickle
import cv2
from os import listdir
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing import image
from tensorflow.keras.utils import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os
import pathlib

EPOCHS = 25
INIT_LR = 1e-3
BS = 32
default_image_size = tuple((256, 256))
image_size = 0
path = pathlib.Path('C:\Dataset1')
width=256
height=256
depth=3

def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None :
            image = cv2.resize(image, default_image_size)   
            return img_to_array(image)
        else :
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None
    
image_list, label_list = [], []
l=[]
try:
    print("[INFO] Loading images ...")
    root_dir = listdir(path)
    for directory in root_dir :
        # remove .DS_Store from list
        if directory == ".DS_Store" :
            root_dir.remove(directory)

    for plant_folder in root_dir :
        plant_disease_folder_list = listdir(f"{path}/{plant_folder}")
        
        for disease_folder in plant_disease_folder_list :
            # remove .DS_Store from list
            if disease_folder == ".DS_Store" :
                plant_disease_folder_list.remove(disease_folder)

        for plant_disease_folder in plant_disease_folder_list:
            print(f"[INFO] Processing {plant_disease_folder} ...")
            plant_disease_image_list = listdir(f"{path}/{plant_folder}/{plant_disease_folder}/")
                
            for single_plant_disease_image in plant_disease_image_list :
                if single_plant_disease_image == ".DS_Store" :
                    plant_disease_image_list.remove(single_plant_disease_image)

            for image in plant_disease_image_list:
                l.append(image)
                image_directory = f"{path}/{plant_folder}/{plant_disease_folder}/{image}"
                if image_directory.endswith(".jpg") == True or image_directory.endswith(".JPG") == True:
                    image_list.append(convert_image_to_array(image_directory))
                    label_list.append(plant_disease_folder)
    print("[INFO] Image loading completed")  
except Exception as e:
    print(f"Error : {e}")
    
label_binarizer = LabelBinarizer()
image_labels = label_binarizer.fit_transform(label_list)
#pickle.dump(label_binarizer,open('label_transform.pkl', 'wb'))
n_classes = len(label_binarizer.classes_)
print()
print( label_binarizer.classes_)

print(image_labels)

np_image_list = np.array(image_list, dtype=np.float16) / 225.0

print()
print("[INFO] Spliting data to train, test")
x_train, x_test, y_train, y_test = train_test_split(np_image_list, image_labels, test_size=0.2, random_state = 42) 

inputShape = (height, width, depth)
chanDim = -1
if K.image_data_format() == "channels_first":
    inputShape = (depth, height, width)
    chanDim = 1

#Create models architecture
model = Sequential()

#Add the first layer
model.add( Conv2D(32, (3,3), activation='relu', input_shape=inputShape))

#Add a pooling layer
model.add(MaxPooling2D(pool_size=(2,2)))

#Add another convolutional layer
model.add( Conv2D(32, (3,3), activation='relu'))

#Add another pooling layer
model.add(MaxPooling2D(pool_size=(2,2)))

#Add a flatenning layer
model.add(Flatten())

#Add a layer with 1000 neurons
model.add(Dense(1000, activation='relu'))

#Add a dropout layer
model.add(Dropout(0.5))

#Add a layer with 500 neurons
model.add(Dense(500, activation='relu'))

#Add a dropout layer
model.add(Dropout(0.5))

#Add a layer with 250 neurons
model.add(Dense(250, activation='relu'))

#Add a dropout layer
model.add(Dropout(0.5))

#Add a layer with 125 neurons
model.add(Dense(125, activation='relu'))

#Add a layer with 10 neurons
model.add(Dense(6, activation='softmax'))

model.compile(loss = 'categorical_crossentropy',
              optimizer = 'adam', metrics = ['accuracy'])

#image_dir="C:\Dataset\plant_folder\Healthy_Leaf\\Healthy_leaf (2).jpg"
image_dir="C:\Dataset\plant_folder\Healthy_Nut\\healthy_nut (8).jpg"
#image_dir="C:\Dataset\plant_folder\Healthy_Trunk\\Healthy (17).jpg"
#image_dir="C:\Dataset\plant_folder\Mahali_Koleroga\\diseases_nut (1).jpg"
#image_dir="C:\Dataset\plant_folder\Stem_bleeding\\diseased (3).jpg"
#image_dir="C:\Dataset\plant_folder\yellow_leaf spot_disease\\yellow_leaf_spot (11).jpeg"
#image_dir = "C://Users//hp//Downloads//Ayurvedic+Healing_Betel+leaf+versus+tobacco.jpg"
im=convert_image_to_array(image_dir)
np_image_li = np.array(im, dtype=np.float16) / 225.0
npp_image = np.expand_dims(np_image_li, axis=0)

result=model.predict(npp_image)
print(result)

import tkinter 
from tkinter import *
root=Tk()
root.title("Arecanut desease detection")
root.geometry("500x300")
#result=model.predict(npp_image)
itemindex = np.where(result==np.max(result))
def getvalu():
    for i in range(len(result[0])):
        re=label_binarizer.classes_[i]+" "+str(round(result[0][i]*100,2))+ " %" 
        print(re)
    if itemindex[1][0]>2:
        re="\nRemedy for the "+ label_binarizer.classes_[itemindex[1][0]]+" :\n",Remedy[itemindex[1][0]]
        print(re)
def show():
    for i in range(len(result[0])):
        re=label_binarizer.classes_[i]+" "+str(round(result[0][i]*100,2))+ " %" 
        print(re)
    if itemindex[1][0]>2:
        re="\nRemedy for the "+ label_binarizer.classes_[itemindex[1][0]]+" :\n",Remedy[itemindex[1][0]]
        print(re)
   # cv2.imshow('input image',img)
    #cv2.imshow('binary image',binary)
    #cv2.imshow('gray image',gray)
    #cv2.imshow('canny image',icanny)
    #cv2.imshow('erosion image',erosion)
    #cv2.imshow('dilation image',dialation)
    #cv2.imshow('edge images',edges)
    cv2.imshow('output image',im)
    
def cancle():
    root.destroy()
Button(text="Show Output",command=getvalu).place(x=10, y=10)
#Label(root, text=re, fg='white', bg='black',font="ar 15 bold").place(x=90, y=200)
Button(text="output with image",command=show).place(x=110, y=10)
Button(text="Cancle",command=cancle).place(x=250, y=10)

root.mainloop()

[INFO] Loading images ...
[INFO] Processing Healthy_Nut ...
[INFO] Processing Mahali_Koleroga ...
[INFO] Image loading completed

['Healthy_Nut' 'Mahali_Koleroga']
[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]]

[INFO] Spliting data to train, test
1/1 [==============================] - 0s 152ms/step
[[0.1496386  0.16285014 0.16287708 0.16832085 0.18967189 0.16664144]]
Healthy_Nut 14.96 %
Mahali_Koleroga 16.29 %


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\User\AppData\Local\Temp\ipykernel_14384\1439347618.py", line 176, in show
    re=label_binarizer.classes_[i]+" "+str(round(result[0][i]*100,2))+ " %"
IndexError: index 2 is out of bounds for axis 0 with size 2
